In [243]:
import pandas as pd
import numpy as np

In [244]:
df_listino = pd.read_csv('data_demo/art_costi.csv')
df_ddt = pd.read_csv('data_demo/ddt_righe.csv')

df_ddt = df_ddt[df_ddt["Data_documento"] > "2024-02-18"]
# remove from listino al the products that are not in the ddt
df_listino = df_listino[df_listino["Codice_articolo"].isin(df_ddt["Codice_articolo"])]

In [245]:
df_listino.head()

,Codice_articolo,Codice_variante,Descrizione,Costo,Data_riferimento,Categoria
26,017.PM152,1,VETRO STRATIFICATO 33.1,30.5,2023-08-28,Vetri
27,017.PM152,2,VETRO STRATIFICATO 33.1,18.0,2023-08-28,Vetri
28,017.PM152,3,VETRO STRATIFICATO 33.1,30.5,2023-08-28,Vetri
29,017.PM152,4,VETRO STRATIFICATO 33.1,18.0,2023-08-28,Vetri
30,017.PM152,5,VETRO STRATIFICATO 33.1,18.0,2023-08-28,Vetri


In [246]:
dict_entities = {}
ids_products_list = []
ids_clienti_list = []
ids_azien_list = []

In [247]:
# prodotti in listino
for index in range(len(df_listino)):
    cod_articolo = df_listino.iloc[index]["Codice_articolo"]
    ids_products_list.append(cod_articolo)
    variante = int(df_listino.iloc[index]["Codice_variante"])
    costo = df_listino.iloc[index]["Costo"]
    data_riferimento = df_listino.iloc[index]["Data_riferimento"]
    categoria = df_listino.iloc[index]["Categoria"]
    descrizione = df_listino.iloc[index]["Descrizione"]
    dict_entities["prod_"+cod_articolo] = {
        "cod_articolo": cod_articolo,
        "variante": variante,
        "costo": costo,
        "data_riferimento": data_riferimento,
        "categoria": categoria,
        "descrizione": descrizione
    }

In [248]:
# prodotti inventati
for index in range(20):
    cod_articolo = "prod_"+str(index)
    ids_products_list.append(cod_articolo)
    variante = int(np.random.choice([0, 1, 2]))
    costo = np.random.uniform(10000, 20000)
    data_riferimento = "2024-03-17"
    categoria = "level2"
    descrizione = "prodotto inventato"
    dict_entities["prod_"+cod_articolo] = {
        "cod_articolo": cod_articolo,
        "variante": variante,
        "costo": costo,
        "data_riferimento": data_riferimento,
        "categoria": categoria,
        "descrizione": descrizione
    }

In [249]:
#fornitori e ddt
for index in range(len(df_ddt)):
    cod_fornitore = int(df_ddt.iloc[index]["COD_CF_Fornitore"])
    dict_entities["fornitore_" + str(cod_fornitore)] = {
        "cod_fornitore": cod_fornitore
    }
    data_documento = df_ddt.iloc[index]["Data_documento"]
    id_documento = df_ddt.iloc[index]["ID_documento"]
    descrizione = df_ddt.iloc[index]["Descrizione"]
    codice_articolo = df_ddt.iloc[index]["Codice_articolo"]
    codice_variante = int(df_ddt.iloc[index]["Codice_variante"])
    codice_categoria = df_ddt.iloc[index]["Codice_categoria"]
    unita_misura = df_ddt.iloc[index]["Unita_misura"]
    quantita_um = df_ddt.iloc[index]["Quantita_um"]
    prezzo_netto_vu = df_ddt.iloc[index]["Prezzo_netto_vu"]
    importo = df_ddt.iloc[index]["importo"]

    if id_documento not in dict_entities:
        dict_entities["documento_" + id_documento] = {
            "data_documento": data_documento,
            "id_documento": id_documento,
            "venduto": [],
            "importo": importo
        }
    dict_entities["documento_" + id_documento]["venduto"].append(
        {
        "codice_articolo": codice_articolo,
        "codice_variante": codice_variante,
        "descrizione": descrizione,
        "codice_categoria": codice_categoria,
        "unita_misura": unita_misura,
        "quantita_um": quantita_um,
        "prezzo_netto_vu": prezzo_netto_vu,
        }
    )

In [250]:
#clienti
for i in range(60):
    dict_entities["cliente_"+str(i)] = { "id_cliente": "cliente_"+str(i)}
    ids_clienti_list.append("cliente_"+str(i))

In [251]:
# aziende
for i in range(3):
    dict_entities["mangini_"+str(i)] = { "id_azienda": "mangini_"+str(i)}
    ids_azien_list.append("mangini_"+str(i))

In [252]:
# preventivi
for i in range(35):
    data_documento =  "2024-03-17"
    id_documento = "preventivo_"+str(i)
    descrizione = "preventivo inventato"
    id_cliente = np.random.choice(ids_clienti_list)
    id_azienda = np.random.choice(ids_azien_list)
    products_list = []
    for j in range(np.random.randint(1, 5)):
        # chose a random product
        codice_articolo = np.random.choice(ids_products_list)
        product = dict_entities["prod_"+codice_articolo]
        codice_variante = int(product["variante"])
        costo = product["costo"]
        categoria = product["categoria"]
        prezzo_netto_vu = costo + np.random.uniform(-0.25, 0.5)
        product_dict = {
            "codice_articolo": codice_articolo,
            "codice_variante": int(codice_variante),
            "codice_categoria": categoria,
            "prezzo_netto_vu": prezzo_netto_vu,
        }
        products_list.append(product_dict)

    importo = sum([product["prezzo_netto_vu"] for product in products_list])

    dict_entities[id_documento] = {
        "data_documento": data_documento,
        "id_documento": id_documento,
        "da_vendere": products_list,
        "importo": importo,
        "id_cliente": id_cliente,
        "id_azienda": id_azienda
        }

In [253]:
# fatture
for i in range(70):
    data_documento =  "2024-03-17"
    id_documento = "fattura_"+str(i)
    descrizione = "fattura inventata"
    id_cliente = np.random.choice(ids_clienti_list)
    id_azienda = np.random.choice(ids_azien_list)
    products_list = []
    for j in range(np.random.randint(1, 5)):
        # chose a random product
        codice_articolo = np.random.choice(ids_products_list)
        product = dict_entities["prod_"+codice_articolo]
        codice_variante = int(product["variante"])
        costo = product["costo"]
        categoria = product["categoria"]
        prezzo_netto_vu = costo + np.random.uniform(-0.25, 0.5)
        product_dict = {
            "codice_articolo": codice_articolo,
            "codice_variante": int(codice_variante),
            "codice_categoria": categoria,
            "prezzo_netto_vu": prezzo_netto_vu,
        }
        products_list.append(product_dict)

    importo = sum([product["prezzo_netto_vu"] for product in products_list])

    dict_entities[id_documento] = {
        "data_documento": data_documento,
        "id_documento": id_documento,
        "venduto": products_list,
        "importo": importo,
        "id_cliente": id_cliente,
        "id_azienda": id_azienda
        }

In [254]:
dict_entities

{'prod_017.PM152': {'cod_articolo': '017.PM152',
  'variante': 0,
  'costo': 10000.0,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO STRATIFICATO 33.1'},
 'prod_090.VST33.SI': {'cod_articolo': '090.VST33.SI',
  'variante': 56,
  'costo': 33.5,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO TRASPARENTE 33.1 SILENT F.G. ANTA INT.'},
 'prod_090.VST33': {'cod_articolo': '090.VST33',
  'variante': 0,
  'costo': 24.5,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO TRASPARENTE STRATIFICATO 3+3 F.G. ANTA INTELAIATA'},
 'prod_090.VST55': {'cod_articolo': '090.VST55',
  'variante': 0,
  'costo': 37.0,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO STRATIFICATO TRASPARENTE 5+5'},
 'prod_090.VST66.SI': {'cod_articolo': '090.VST66.SI',
  'variante': 0,
  'costo': 51.5,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO T

In [255]:
keys = dict_entities.keys()
keys = sorted(keys)
keys

['cliente_0',
 'cliente_1',
 'cliente_10',
 'cliente_11',
 'cliente_12',
 'cliente_13',
 'cliente_14',
 'cliente_15',
 'cliente_16',
 'cliente_17',
 'cliente_18',
 'cliente_19',
 'cliente_2',
 'cliente_20',
 'cliente_21',
 'cliente_22',
 'cliente_23',
 'cliente_24',
 'cliente_25',
 'cliente_26',
 'cliente_27',
 'cliente_28',
 'cliente_29',
 'cliente_3',
 'cliente_30',
 'cliente_31',
 'cliente_32',
 'cliente_33',
 'cliente_34',
 'cliente_35',
 'cliente_36',
 'cliente_37',
 'cliente_38',
 'cliente_39',
 'cliente_4',
 'cliente_40',
 'cliente_41',
 'cliente_42',
 'cliente_43',
 'cliente_44',
 'cliente_45',
 'cliente_46',
 'cliente_47',
 'cliente_48',
 'cliente_49',
 'cliente_5',
 'cliente_50',
 'cliente_51',
 'cliente_52',
 'cliente_53',
 'cliente_54',
 'cliente_55',
 'cliente_56',
 'cliente_57',
 'cliente_58',
 'cliente_59',
 'cliente_6',
 'cliente_7',
 'cliente_8',
 'cliente_9',
 'documento_2024-BADDV-0000270',
 'documento_2024-BADDV-0000273',
 'documento_2024-BADDV-0000275',
 'documento

In [256]:
import json
with open("dict_entities.json", "w") as fp:
    json.dump(dict_entities , fp) 

In [257]:
entities_types = ["prod", "fornitore", "documento", "cliente", "mangini", "preventivo", "fattura"]
dict_types_list = {"prod":[], "fornitore":[], "documento":[], "cliente":[], "mangini":[], "preventivo":[], "fattura":[]}
for key in dict_entities:
    key_first_part = key.split("_")[0]
    dict_types_list[key_first_part].append(dict_entities[key])

In [258]:
for key in entities_types:
    df = pd.DataFrame(dict_types_list[key])
    df.to_csv("data_demo/df_"+key+".csv", index=False)

In [259]:
df_prod = pd.DataFrame(dict_types_list["fattura"])
df_prod

,data_documento,id_documento,venduto,importo,id_cliente,id_azienda
0,2024-03-17,fattura_0,"[{'codice_articolo': '052.MP.VT.POR.TR.45.04',...",10112.532798,cliente_47,mangini_1
1,2024-03-17,fattura_1,"[{'codice_articolo': 'prod_1', 'codice_variant...",30066.918677,cliente_25,mangini_1
2,2024-03-17,fattura_2,"[{'codice_articolo': '017.PM152', 'codice_vari...",10003.409174,cliente_35,mangini_2
3,2024-03-17,fattura_3,"[{'codice_articolo': 'prod_3', 'codice_variant...",36963.149399,cliente_12,mangini_1
4,2024-03-17,fattura_4,"[{'codice_articolo': '017.PM006.08', 'codice_v...",10001.923832,cliente_12,mangini_0
...,...,...,...,...,...,...
65,2024-03-17,fattura_65,"[{'codice_articolo': '017.PM152', 'codice_vari...",10014.060354,cliente_42,mangini_0
66,2024-03-17,fattura_66,"[{'codice_articolo': '090.VST33', 'codice_vari...",10072.101658,cliente_8,mangini_0
67,2024-03-17,fattura_67,"[{'codice_articolo': '017.PM152', 'codice_vari...",10000.091446,cliente_23,mangini_0
68,2024-03-17,fattura_68,"[{'codice_articolo': '090.VST33', 'codice_vari...",10025.070901,cliente_55,mangini_1
